In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import tensorflow.keras as keras

In [2]:
df = pd.read_csv('/content/diabetes.csv')

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.isnull().sum()

,0
Pregnancies,0
Glucose,0
BloodPressure,0
SkinThickness,0
Insulin,0
BMI,0
DiabetesPedigreeFunction,0
Age,0
Outcome,0


In [7]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [8]:
x = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [9]:
x.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


In [10]:
y.head()

,Outcome
0,1
1,0
2,1
3,0
4,1


# Preprocessing

In [11]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [12]:
# Split the data intp training and test
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

## Building MLP Model

In [13]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [16]:
model = Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [28]:
model.fit(x_train,y_train,epochs=20,batch_size=32, validation_data=(x_test,y_test))

Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7792 - loss: 0.4408 - val_accuracy: 0.8052 - val_loss: 0.4533
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7936 - loss: 0.4168 - val_accuracy: 0.8052 - val_loss: 0.4522
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8188 - loss: 0.4035 - val_accuracy: 0.7987 - val_loss: 0.4512
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7989 - loss: 0.4081 - val_accuracy: 0.7987 - val_loss: 0.4516
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7950 - loss: 0.4359 - val_accuracy: 0.7987 - val_loss: 0.4533
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8196 - loss: 0.4080 - val_accuracy: 0.8117 - val_loss: 0.4520
Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7908 - loss: 0.4249 - val_accuracy: 0.8052 - val_loss: 0.4531
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8089 - loss: 0.4233 - val_accuracy: 0.8052 - val_loss

# HyperParameter Tuning

# 1) HOW TO SELECT appropriate optimizer
# 2) no. of in layer
# 3)

In [19]:
!pip install -U keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 10.2 MB/s eta 0:00:00


In [29]:
import kerastuner as kt

In [30]:
# Hyper paramater for optimization
def build_model(hp):
  model = keras.Sequential()
  model.add(keras.layers.Dense(32,activation='relu',input_dim=8))
  model.add(keras.layers.Dense(1,activation='sigmoid'))
  optimizer = hp.Choice('optimizer',values=['adam','sgd','rmsprop', 'adadelta'])
  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [31]:
# Best Optimization
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [32]:
tuner

In [33]:
tuner.search(x_train,y_train,epochs=10,validation_data=(x_test,y_test))

In [34]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

# No.of Layers

In [35]:
def build_model(hp):
  model = Sequential()
  units = hp.Int('units',min_value=8,max_value=256,step=8)
  model.add(Dense(units=units,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [36]:
# Tunner Object
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5, directory='mydir', project_name='myproject')

In [37]:
tuner.search(x_train,y_train,epochs=10,validation_data=(x_test,y_test))

Trial 5 Complete [00h 00m 05s]
val_accuracy: 0.8246753215789795

Best val_accuracy So Far: 0.8246753215789795
Total elapsed time: 00h 00m 22s


In [38]:
tuner.get_best_hyperparameters()[0].values

{'units': 168}

# How many Hidden Layer Requried

In [40]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(168,activation='relu',input_dim=8))
  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    model.add(Dense(168,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [41]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5, directory='mydir', project_name='num_layers')

In [42]:
tuner.search(x_train,y_train,epochs=10,validation_data=(x_test,y_test))

Trial 5 Complete [00h 00m 05s]
val_accuracy: 0.8051947951316833

Best val_accuracy So Far: 0.8181818127632141
Total elapsed time: 00h 00m 29s


In [43]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 6}

In [44]:
# All HyperParameter at one go

In [46]:
from typing import Counter
def build_model(hp):
  model = Sequential()
  Counter=0

  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    if Counter==0:
      model.add(Dense(hp.Int('units'+str(i),min_value=8,max_value=256,step=8),
                      activation=hp.Choice('activation'+str(i), values=['relu','tanh','sigmoid', 'softmax','leaky_relu']),input_dim=8))
      model.add(Dropout(hp.Choice('dropout'+str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7])))
    else:
      model.add(Dense(hp.Int('units'+str(i),min_value=8,max_value=256,step=8),
                      activation=hp.Choice('activation'+str(i), values=['relu','tanh','sigmoid', 'softmax','leaky_relu'])))
      model.add(Dropout(hp.Choice('dropout'+str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7])))
    Counter+=1
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer=hp.Choice('optimizer',values=['adam','sgd','rmsprop', 'adadelta']),
                loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [49]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5, directory='mymodel', project_name='finailHyperparamter')

In [50]:
tuner.search(x_train,y_train,epochs=10,validation_data=(x_test,y_test))

Trial 5 Complete [00h 00m 13s]
val_accuracy: 0.7597402334213257

Best val_accuracy So Far: 0.8311688303947449
Total elapsed time: 00h 00m 44s


In [53]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 1,
 'units0': 120,
 'activation0': 'relu',
 'dropout0': 0.3,
 'optimizer': 'rmsprop',
 'units1': 72,
 'activation1': 'sigmoid',
 'dropout1': 0.3,
 'units2': 160,
 'activation2': 'relu',
 'dropout2': 0.3,
 'units3': 240,
 'activation3': 'tanh',
 'dropout3': 0.5,
 'units4': 8,
 'activation4': 'tanh',
 'dropout4': 0.2}